Loading Libraries


In [1]:
!pip install keras-tuner

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras_tuner as kt

Load & preprocess CIFAR‑10

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
y_train = utils.to_categorical(y_train, 10)
y_test  = utils.to_categorical(y_test, 10)

MixUp data generator

In [4]:
def mixup_generator(x, y, batch_size=128, alpha=0.2):
    n = x.shape[0]
    idx = np.arange(n)
    while True:
        np.random.shuffle(idx)
        for i in range(0, n, batch_size):
            batch_idx = idx[i : i + batch_size]
            x1, y1 = x[batch_idx].copy(), y[batch_idx].copy()
            lam = np.random.beta(alpha, alpha, size=len(x1))
            lam_x = lam.reshape(-1,1,1,1)
            lam_y = lam.reshape(-1,1)
            idx2 = np.random.choice(n, size=len(x1), replace=False)
            x2, y2 = x[idx2], y[idx2]
            x_batch = lam_x * x1 + (1 - lam_x) * x2
            y_batch = lam_y * y1 + (1 - lam_y) * y2
            yield x_batch, y_batch


Build tunable model using ResNet50 backbone

In [5]:
def build_model(hp):
    # Backbone
    base = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(32,32,3),
        pooling='avg'
    )
    base.trainable = False

    inputs = layers.Input(shape=(32,32,3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base(x, training=False)

    # Tunable dense layer size
    units = hp.Choice('dense_units', [128, 256, 512], default=256)
    x = layers.Dense(units, activation='relu')(x)
    x = layers.Dropout(
        hp.Float('dropout_rate', min_value=0.3, max_value=0.6, step=0.1, default=0.5)
    )(x)

    outputs = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, outputs)

    # Compile with tunable learning rate and label smoothing
    lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=1e-3)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=loss,
        metrics=['accuracy']
    )
    return model

Set up Keras Tuner (Hyperband)

In [6]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='cifar5_tuner',
    project_name='episode5'
)

# Early stopping during tuning
stop_early = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

Reloading Tuner from cifar5_tuner\episode5\tuner0.json


Run hyperparameter search

In [7]:
batch_size = 128
steps_per_epoch = x_train.shape[0] // batch_size

tuner.search(
    mixup_generator(x_train, y_train, batch_size=batch_size, alpha=0.2),
    steps_per_epoch=steps_per_epoch,
    validation_data=(x_test, y_test),
    epochs=20,
    callbacks=[stop_early]
)

Trial 30 Complete [00h 15m 40s]
val_accuracy: 0.17579999566078186

Best val_accuracy So Far: 0.3617999851703644
Total elapsed time: 1d 08h 42m 04s


Retrieve the best model

In [8]:
best_model = tuner.get_best_models(num_models=1)[0]

Fine‑tune

In [10]:
fine_tune_history = best_model.fit(
    mixup_generator(x_train, y_train, batch_size=batch_size, alpha=0.2),
    steps_per_epoch=steps_per_epoch,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
    ]
)


Epoch 1/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 95s 235ms/step - accuracy: 0.3290 - loss: 1.9948 - val_accuracy: 0.3576 - val_loss: 1.9121 - learning_rate: 1.3434e-04
Epoch 2/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 95s 243ms/step - accuracy: 0.3324 - loss: 1.9926 - val_accuracy: 0.3544 - val_loss: 1.9187 - learning_rate: 1.3434e-04
Epoch 3/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 91s 233ms/step - accuracy: 0.3376 - loss: 1.9923 - val_accuracy: 0.3628 - val_loss: 1.9051 - learning_rate: 1.3434e-04
Epoch 4/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 92s 235ms/step - accuracy: 0.3354 - loss: 1.9974 - val_accuracy: 0.3441 - val_loss: 1.9216 - learning_rate: 1.3434e-04
Epoch 5/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 91s 234ms/step - accuracy: 0.3365 - loss: 1.9883 - val_accuracy: 0.3662 - val_loss: 1.8919 - learning_rate: 1.3434e-04
Epoch 6/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 92s 236ms/step - accuracy: 0.3391 - loss: 1.9894 - val_accuracy: 0.3636 - val_loss: 1.8988 - learning_rate: 1.3434e-04
Epoch 7/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 94s 24

Final evaluation

In [11]:
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=2)
print(f"Final test accuracy: {test_acc:.4f}")

313/313 - 24s - 75ms/step - accuracy: 0.3764 - loss: 1.8865
Final test accuracy: 0.3764
